<a href="https://colab.research.google.com/github/armanfeili/novartis_datathon_2025/blob/Arman/notebooks/colab/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧬 Novartis Datathon 2025 - Complete Training Pipeline

This notebook provides a **complete end-to-end pipeline** for the Novartis Datathon 2025:

1. **🔧 Environment Setup** - Mount Drive, clone repo, install dependencies
2. **📊 Data Loading** - Load raw data and build panels
3. **🔬 Feature Engineering** - Build scenario-specific features  
4. **🏋️ Model Training** - Train CatBoost models with cross-validation
5. **📈 Evaluation** - Analyze model performance
6. **📤 Submission** - Generate competition submission files

---

**Quick Start:**
1. Run all cells in order
2. Data should be placed in `data/raw/TRAIN/` and `data/raw/TEST/`
3. Trained models and submissions are saved to Google Drive

---

## 1. Environment Setup

In [ ]:
# ==============================================================================
# 1.1 Detect Environment and Mount Google Drive
# ==============================================================================
import sys
import os
from pathlib import Path

# Detect if running in Colab
IN_COLAB = 'google.colab' in sys.modules

print(f"🖥️  Environment: {'Google Colab' if IN_COLAB else 'Local'}")
print(f"🐍 Python: {sys.version.split()[0]}")

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    print("✅ Google Drive mounted successfully")
else:
    print("⚠️ Not running in Colab - using local paths")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive mounted successfully


In [ ]:
# ==============================================================================
# 1.2 Clone Repository and Set Paths
# ==============================================================================
import os

# --- Configuration (MODIFY THESE) ---
REPO_URL = "https://github.com/armanfeili/novartis_datathon_2025.git"
BRANCH = "Arman"  # Change to your working branch

# Paths depend on environment
if IN_COLAB:
    DRIVE_BASE = "/content/drive/MyDrive"
    PROJECT_PATH = "/content/novartis_datathon_2025"  # Clone to /content for speed
    DATA_PATH = f"{DRIVE_BASE}/novartis-datathon-2025/data"  # Data on Drive
    ARTIFACTS_PATH = f"{DRIVE_BASE}/novartis-datathon-2025/artifacts"
    SUBMISSIONS_PATH = f"{DRIVE_BASE}/novartis-datathon-2025/submissions"
else:
    # Local paths (relative to notebook location)
    PROJECT_PATH = str(Path.cwd().parent.parent)
    DATA_PATH = os.path.join(PROJECT_PATH, "data")
    ARTIFACTS_PATH = os.path.join(PROJECT_PATH, "artifacts")
    SUBMISSIONS_PATH = os.path.join(PROJECT_PATH, "submissions")
# --------------------------------

if IN_COLAB:
    # Clone or update repository
    if not os.path.exists(PROJECT_PATH):
        print(f"📥 Cloning repository...")
        !git clone --branch {BRANCH} {REPO_URL} {PROJECT_PATH}
    else:
        print(f"📂 Repository exists. Pulling latest changes...")
        %cd {PROJECT_PATH}
        !git fetch origin {BRANCH}
        !git reset --hard origin/{BRANCH}
    
    %cd {PROJECT_PATH}
    
    # Create symlinks to Drive data (if data is on Drive)
    if os.path.exists(DATA_PATH):
        local_data = os.path.join(PROJECT_PATH, "data")
        if not os.path.exists(local_data):
            !ln -s {DATA_PATH} {local_data}
            print(f"🔗 Linked data directory from Drive")

# Create required directories
for path in [DATA_PATH, ARTIFACTS_PATH, SUBMISSIONS_PATH]:
    os.makedirs(path, exist_ok=True)

# Print paths
print(f"\n📁 Project: {PROJECT_PATH}")
print(f"📁 Data: {DATA_PATH}")
print(f"📁 Artifacts: {ARTIFACTS_PATH}")
print(f"📁 Submissions: {SUBMISSIONS_PATH}")

📂 Repository exists at /content/drive/MyDrive/novartis_datathon_2025. Pulling latest changes...
/content/drive/MyDrive/novartis_datathon_2025
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 10 (delta 6), reused 10 (delta 6), pack-reused 0 (from 0)
Unpacking objects: 100% (10/10), 8.06 KiB | 26.00 KiB/s, done.
From https://github.com/armanfeili/novartis_datathon_2025
 * branch            Arman      -> FETCH_HEAD
   67c14aa..5c33709  Arman      -> origin/Arman
HEAD is now at 5c33709 project setup - 3
/content/drive/MyDrive/novartis_datathon_2025

📁 Project Path: /content/drive/MyDrive/novartis_datathon_2025
📁 Data Path: /content/drive/MyDrive/novartis-datathon-2025/data
📁 Artifacts Path: /content/drive/MyDrive/novartis-datathon-2025/artifacts
📁 Submissions Path: /content/drive/MyDrive/novartis-datathon-2025/submissions


In [ ]:
# ==============================================================================
# 1.3 Install Dependencies
# ==============================================================================
import subprocess

print("📦 Installing dependencies...")

# Install from colab requirements
!pip install -q -r env/colab_requirements.txt

# Verify key packages
import importlib

packages = [
    ('numpy', 'numpy'),
    ('pandas', 'pandas'),
    ('sklearn', 'scikit-learn'),
    ('yaml', 'pyyaml'),
    ('tqdm', 'tqdm'),
    ('catboost', 'catboost'),
    ('lightgbm', 'lightgbm'),
    ('xgboost', 'xgboost'),
    ('pyarrow', 'pyarrow'),
]

print("\n📋 Package Status:")
for import_name, pkg_name in packages:
    try:
        mod = importlib.import_module(import_name)
        version = getattr(mod, '__version__', 'installed')
        print(f"  ✅ {pkg_name}: {version}")
    except ImportError:
        print(f"  ❌ {pkg_name}: not installed")

print("\n✅ Dependencies installed!")

📦 Installing dependencies...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 63.0 MB/s eta 0:00:00
  ✅ torch
  ✅ numpy
  ✅ pandas
  ✅ lightgbm
  ✅ xgboost
  ✅ catboost
  ✅ sklearn
  ✅ yaml

✅ All dependencies installed!


## 2. Import Modules and Verify Environment

Import project modules and verify GPU availability.

In [ ]:
# ==============================================================================
# 2.1 Import Project Modules
# ==============================================================================
import sys
import os
import gc
import warnings
warnings.filterwarnings('ignore')

# Ensure project root is in path
if PROJECT_PATH not in sys.path:
    sys.path.insert(0, PROJECT_PATH)

# Standard imports
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime

# Project imports
from src.utils import (
    load_config, set_seed, setup_logging, timer, 
    get_device, get_gpu_info, print_environment_info,
    clear_memory, get_memory_usage, optimize_dataframe_memory
)
from src.data import (
    get_panel, load_raw_data, prepare_base_panel, 
    compute_pre_entry_stats, handle_missing_values,
    META_COLS
)
from src.features import (
    get_features, make_features, split_features_target_meta,
    get_feature_columns, SCENARIO_CONFIG
)
from src.train import train_scenario_model, run_cross_validation
from src.evaluate import compute_metric1, compute_metric2, compute_per_series_error
from src.inference import (
    generate_submission, detect_test_scenarios, 
    validate_submission_format, save_submission_with_versioning
)

print("✅ All modules imported successfully!")

# ==============================================================================
# 2.2 Display Environment Information
# ==============================================================================
print_environment_info()

🖥️  Device: cpu

✅ All modules imported successfully!


## 3. Load Configuration and Set Seed

Load all configuration files and set random seed for reproducibility.

In [ ]:
# ==============================================================================
# 3.1 Load Configurations
# ==============================================================================
data_config = load_config('configs/data.yaml')
features_config = load_config('configs/features.yaml')
run_config = load_config('configs/run_defaults.yaml')
model_config = load_config('configs/model_cat.yaml')  # Hero model: CatBoost

# Set random seed for reproducibility
SEED = run_config['reproducibility']['seed']
set_seed(SEED)

# Setup logging
setup_logging(level=run_config.get('logging', {}).get('level', 'INFO'))

print("📋 Configurations loaded:")
print(f"  - Random seed: {SEED}")
print(f"  - Scenarios: {list(run_config['scenarios'].keys())}")
print(f"  - Model: {model_config.get('model_type', 'catboost')}")

# Display scenario details
print(f"\n📅 Scenario Configuration:")
for s_name, s_config in run_config['scenarios'].items():
    print(f"  {s_name}:")
    print(f"    Forecast: months {s_config['forecast_start']} to {s_config['forecast_end']}")
    print(f"    Feature cutoff: month {s_config['feature_cutoff']}")

📋 Configurations loaded:
  - Data config: ['drive', 'local', 'files', 'keys', 'dates', 'columns', 'validation']
  - Features config: ['feature_groups', 'lags', 'rolling', 'diff', 'time_features', 'interactions', 'selection', 'encoding']
  - Run config: ['experiment', 'run', 'reproducibility', 'cv', 'paths', 'output', 'metrics', 'logging', 'drive', 'hardware']
  - Model configs: ['lightgbm', 'xgboost', 'catboost', 'linear', 'neural_network']

🎲 Random seed: 42


## 4. Load and Explore Data

Load the training and test data panels.

In [ ]:
# ==============================================================================
# 4.1 Load Training Panel
# ==============================================================================
print("📂 Loading training data...")

with timer("Load train panel"):
    train_panel = get_panel(split='train', config=data_config, use_cache=True)

# Display statistics
n_series = train_panel[['country', 'brand_name']].drop_duplicates().shape[0]
print(f"\n📊 Training Panel Statistics:")
print(f"  Shape: {train_panel.shape[0]:,} rows × {train_panel.shape[1]} columns")
print(f"  Unique series: {n_series:,}")
print(f"  Time range: {train_panel['months_postgx'].min()} to {train_panel['months_postgx'].max()}")

# Bucket distribution
bucket_dist = train_panel[['country', 'brand_name', 'bucket']].drop_duplicates()['bucket'].value_counts()
print(f"\n🪣 Bucket Distribution:")
for bucket, count in bucket_dist.items():
    pct = count / n_series * 100
    print(f"  Bucket {bucket}: {count:,} series ({pct:.1f}%)")

# Memory usage
mem_mb = train_panel.memory_usage(deep=True).sum() / (1024**2)
print(f"\n💾 Memory: {mem_mb:.1f} MB")

📂 Data Directories:
  Raw: /content/drive/MyDrive/novartis-datathon-2025/data/raw (exists: True)
  Interim: /content/drive/MyDrive/novartis-datathon-2025/data/interim (exists: True)
  Processed: /content/drive/MyDrive/novartis-datathon-2025/data/processed (exists: True)

📄 Available raw files (0):


In [ ]:
# ==============================================================================
# 4.2 Load Test Panel
# ==============================================================================
print("📂 Loading test data...")

with timer("Load test panel"):
    test_panel = get_panel(split='test', config=data_config, use_cache=True)

# Detect scenarios
test_scenarios = detect_test_scenarios(test_panel)
n_test_series = test_panel[['country', 'brand_name']].drop_duplicates().shape[0]

print(f"\n📊 Test Panel Statistics:")
print(f"  Shape: {test_panel.shape[0]:,} rows × {test_panel.shape[1]} columns")
print(f"  Unique series: {n_test_series:,}")
print(f"  Scenario 1 series: {len(test_scenarios[1]):,}")
print(f"  Scenario 2 series: {len(test_scenarios[2]):,}")

# Clear memory
clear_memory()
print(f"\n🧹 Memory cleared")

AttributeError: 'NoneType' object has no attribute 'items'

In [ ]:
# ==============================================================================
# 4.3 Quick Data Exploration
# ==============================================================================
import matplotlib.pyplot as plt

# Set up plotting
plt.style.use('default')
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. y_norm distribution
ax = axes[0, 0]
train_panel['y_norm'].hist(bins=50, ax=ax, color='steelblue', edgecolor='white')
ax.axvline(x=1.0, color='red', linestyle='--', label='No erosion (1.0)')
ax.axvline(x=0.25, color='orange', linestyle='--', label='Bucket 1 threshold')
ax.set_xlabel('Normalized Volume (y_norm)')
ax.set_ylabel('Frequency')
ax.set_title('Distribution of y_norm')
ax.legend()

# 2. Mean erosion curve by bucket
ax = axes[0, 1]
for bucket in [1, 2]:
    bucket_data = train_panel[train_panel['bucket'] == bucket]
    erosion_by_month = bucket_data.groupby('months_postgx')['y_norm'].mean()
    ax.plot(erosion_by_month.index, erosion_by_month.values, 
            label=f'Bucket {bucket}', linewidth=2)
ax.axhline(y=1.0, color='gray', linestyle=':', alpha=0.7)
ax.set_xlabel('Months Post Generic Entry')
ax.set_ylabel('Mean Normalized Volume')
ax.set_title('Erosion Curves by Bucket')
ax.legend()
ax.grid(True, alpha=0.3)

# 3. Number of generics over time
ax = axes[1, 0]
ngxs_by_month = train_panel.groupby('months_postgx')['n_gxs'].mean()
ax.bar(ngxs_by_month.index, ngxs_by_month.values, color='forestgreen', alpha=0.7)
ax.set_xlabel('Months Post Generic Entry')
ax.set_ylabel('Mean Number of Generics')
ax.set_title('Average Generic Competition Over Time')
ax.grid(True, alpha=0.3, axis='y')

# 4. Hospital rate distribution
ax = axes[1, 1]
if 'hospital_rate' in train_panel.columns:
    hr_by_series = train_panel.groupby(['country', 'brand_name'])['hospital_rate'].first()
    hr_by_series.hist(bins=30, ax=ax, color='purple', edgecolor='white', alpha=0.7)
    ax.set_xlabel('Hospital Rate (%)')
    ax.set_ylabel('Number of Series')
    ax.set_title('Hospital Rate Distribution')

plt.tight_layout()
plt.show()

print("✅ Data exploration complete")

## 5. Feature Engineering

Build scenario-specific features for training.

In [ ]:
# ==============================================================================
# 5.1 Build Features for Both Scenarios
# ==============================================================================

# Build Scenario 1 features (forecast months 0-23 using pre-entry only)
print("🔬 Building Scenario 1 features...")
with timer("Scenario 1 features"):
    X_train_s1, y_train_s1, meta_train_s1 = get_features(
        split='train', scenario=1, mode='train',
        data_config=data_config, features_config=features_config,
        use_cache=True
    )
print(f"  X shape: {X_train_s1.shape}")
print(f"  y shape: {y_train_s1.shape}")
print(f"  Features: {len(X_train_s1.columns)}")

# Build Scenario 2 features (forecast months 6-23 using pre-entry + months 0-5)
print("\n🔬 Building Scenario 2 features...")
with timer("Scenario 2 features"):
    X_train_s2, y_train_s2, meta_train_s2 = get_features(
        split='train', scenario=2, mode='train',
        data_config=data_config, features_config=features_config,
        use_cache=True
    )
print(f"  X shape: {X_train_s2.shape}")
print(f"  y shape: {y_train_s2.shape}")
print(f"  Features: {len(X_train_s2.columns)}")

# Display some feature examples
print(f"\n📋 Sample Features (Scenario 1):")
print(f"  {list(X_train_s1.columns[:10])}...")

# Check for early erosion features in S2 only
s2_only_features = [c for c in X_train_s2.columns if 'erosion_0' in c or 'avg_vol_0' in c]
if s2_only_features:
    print(f"\n📋 Scenario 2 Specific Features:")
    print(f"  {s2_only_features[:5]}...")

clear_memory()

## 6. Model Training

Train CatBoost models for both scenarios using cross-validation.

In [ ]:
# ==============================================================================
# 6.1 Training Configuration
# ==============================================================================

# Training settings
N_FOLDS = 5
MODEL_TYPE = 'catboost'  # Options: catboost, lightgbm, xgboost, linear

# Create run ID
RUN_ID = datetime.now().strftime("%Y%m%d_%H%M%S")
RUN_DIR = Path(ARTIFACTS_PATH) / RUN_ID
RUN_DIR.mkdir(parents=True, exist_ok=True)

print(f"🏃 Training Configuration:")
print(f"  Run ID: {RUN_ID}")
print(f"  Model: {MODEL_TYPE}")
print(f"  CV Folds: {N_FOLDS}")
print(f"  Artifacts: {RUN_DIR}")

# Check if GPU is available for CatBoost
gpu_info = get_gpu_info()
if gpu_info['gpu_available'] and gpu_info.get('cuda_version'):
    print(f"  🚀 GPU: {gpu_info['device_name']} - will use GPU training")
    use_gpu = True
else:
    print(f"  💻 Using CPU training")
    use_gpu = False

In [ ]:
# ==============================================================================
# 6.2 Train Scenario 1 Model
# ==============================================================================
from src.models import get_model_class
from src.train import compute_sample_weights
from src.validation import get_fold_series

print("🏋️ Training Scenario 1 model...")
print("=" * 60)

# Get model class and config
ModelClass = get_model_class(MODEL_TYPE)

# Prepare model config with optional GPU
s1_model_config = model_config.copy()
if use_gpu and MODEL_TYPE == 'catboost':
    s1_model_config['params'] = {**s1_model_config.get('params', {}), **{'task_type': 'GPU', 'devices': '0'}}

# Train with cross-validation
s1_cv_results = run_cross_validation(
    X=X_train_s1,
    y=y_train_s1,
    meta_df=meta_train_s1,
    scenario=1,
    model_config=s1_model_config,
    run_config=run_config,
    n_folds=N_FOLDS,
    save_dir=RUN_DIR / 'models_s1',
    run_id=RUN_ID,
)

print(f"\n✅ Scenario 1 Training Complete")
print(f"  Mean CV Metric: {s1_cv_results['mean_score']:.6f} ± {s1_cv_results['std_score']:.6f}")

# Save S1 OOF predictions
oof_s1 = pd.DataFrame({
    'y_true': y_train_s1,
    'y_pred': s1_cv_results['oof_predictions'],
})
oof_s1.to_csv(RUN_DIR / 'oof_s1.csv', index=False)

clear_memory()

### 6.3 Train Scenario 2 Model

In [ ]:
# ==============================================================================
# 6.3 Train Scenario 2 Model
# ==============================================================================
print("🏋️ Training Scenario 2 model...")
print("=" * 60)

# Prepare model config
s2_model_config = model_config.copy()
if use_gpu and MODEL_TYPE == 'catboost':
    s2_model_config['params'] = {**s2_model_config.get('params', {}), **{'task_type': 'GPU', 'devices': '0'}}

# Train with cross-validation
s2_cv_results = run_cross_validation(
    X=X_train_s2,
    y=y_train_s2,
    meta_df=meta_train_s2,
    scenario=2,
    model_config=s2_model_config,
    run_config=run_config,
    n_folds=N_FOLDS,
    save_dir=RUN_DIR / 'models_s2',
    run_id=RUN_ID,
)

print(f"\n✅ Scenario 2 Training Complete")
print(f"  Mean CV Metric: {s2_cv_results['mean_score']:.6f} ± {s2_cv_results['std_score']:.6f}")

# Save S2 OOF predictions
oof_s2 = pd.DataFrame({
    'y_true': y_train_s2,
    'y_pred': s2_cv_results['oof_predictions'],
})
oof_s2.to_csv(RUN_DIR / 'oof_s2.csv', index=False)

clear_memory()

# Summary
print("\n" + "=" * 60)
print("📊 TRAINING SUMMARY")
print("=" * 60)
print(f"  Scenario 1 CV: {s1_cv_results['mean_score']:.6f} ± {s1_cv_results['std_score']:.6f}")
print(f"  Scenario 2 CV: {s2_cv_results['mean_score']:.6f} ± {s2_cv_results['std_score']:.6f}")
print(f"  Models saved to: {RUN_DIR}")

## 7. Generate Submission

Generate predictions on test data and create submission files.

In [ ]:
# ==============================================================================
# 7.1 Build Test Features and Generate Predictions
# ==============================================================================
import joblib

print("📤 Generating submission...")

# Build test features for Scenario 1
print("  Building S1 test features...")
X_test_s1, _, meta_test_s1 = get_features(
    split='test', scenario=1, mode='test',
    data_config=data_config, features_config=features_config,
    use_cache=True
)

# Build test features for Scenario 2  
print("  Building S2 test features...")
X_test_s2, _, meta_test_s2 = get_features(
    split='test', scenario=2, mode='test',
    data_config=data_config, features_config=features_config,
    use_cache=True
)

# Load trained models and make predictions
print("  Loading models and predicting...")

# Scenario 1 predictions (average across folds)
s1_preds_list = []
s1_model_dir = RUN_DIR / 'models_s1'
for fold_path in sorted(s1_model_dir.glob('fold_*')):
    model_path = fold_path / 'model.cbm'  # CatBoost format
    if model_path.exists():
        model = ModelClass.load(str(model_path), s1_model_config)
        preds = model.predict(X_test_s1)
        s1_preds_list.append(preds)

s1_test_preds = np.mean(s1_preds_list, axis=0) if s1_preds_list else np.ones(len(X_test_s1))

# Scenario 2 predictions (average across folds)
s2_preds_list = []
s2_model_dir = RUN_DIR / 'models_s2'
for fold_path in sorted(s2_model_dir.glob('fold_*')):
    model_path = fold_path / 'model.cbm'
    if model_path.exists():
        model = ModelClass.load(str(model_path), s2_model_config)
        preds = model.predict(X_test_s2)
        s2_preds_list.append(preds)

s2_test_preds = np.mean(s2_preds_list, axis=0) if s2_preds_list else np.ones(len(X_test_s2))

print(f"  S1 predictions: {len(s1_test_preds):,}")
print(f"  S2 predictions: {len(s2_test_preds):,}")

### 7.2 Create Submission File

In [ ]:
# ==============================================================================
# 7.2 Create and Save Submission
# ==============================================================================

# Create submission dataframes
submission_s1 = meta_test_s1[['country', 'brand_name', 'months_postgx']].copy()
submission_s1['volume'] = s1_test_preds * meta_test_s1['avg_vol_12m'].values  # Convert y_norm to volume

submission_s2 = meta_test_s2[['country', 'brand_name', 'months_postgx']].copy()
submission_s2['volume'] = s2_test_preds * meta_test_s2['avg_vol_12m'].values

# Combine submissions
submission = pd.concat([submission_s1, submission_s2], ignore_index=True)

# Clip negative volumes to 0
submission['volume'] = submission['volume'].clip(lower=0)

# Validate submission format
is_valid, issues = validate_submission_format(submission)
if is_valid:
    print("✅ Submission format validated")
else:
    print(f"⚠️ Validation issues: {issues}")

# Save submission
submission_path = Path(SUBMISSIONS_PATH) / f"submission_{RUN_ID}.csv"
submission.to_csv(submission_path, index=False)

print(f"\n📄 Submission saved to: {submission_path}")
print(f"  Shape: {submission.shape}")
print(f"  Columns: {list(submission.columns)}")

# Statistics
print(f"\n📊 Submission Statistics:")
print(f"  Volume min: {submission['volume'].min():.2f}")
print(f"  Volume max: {submission['volume'].max():.2f}")
print(f"  Volume mean: {submission['volume'].mean():.2f}")
print(f"  Volume median: {submission['volume'].median():.2f}")

# Preview
print(f"\n📋 Preview:")
display(submission.head(10))

## 10. Utilities & Helpers

Useful utility functions for common operations.

In [ ]:
# ==============================================================================
# 7.3 Download Submission (Colab only)
# ==============================================================================
if IN_COLAB:
    print("📥 Downloading submission file...")
    from google.colab import files
    files.download(str(submission_path))
    print("✅ Download complete!")
else:
    print(f"📄 Submission available at: {submission_path}")

# Also sync to Drive if in Colab
if IN_COLAB:
    # Copy to Drive submissions folder
    import shutil
    drive_submission_path = f"{SUBMISSIONS_PATH}/submission_{RUN_ID}.csv"
    shutil.copy(str(submission_path), drive_submission_path)
    print(f"☁️ Saved to Google Drive: {drive_submission_path}")

## 8. Utilities

Helper functions for common operations.

In [ ]:
# ==============================================================================
# 8.1 Utility Functions
# ==============================================================================

def show_memory():
    """Display current memory usage."""
    mem = get_memory_usage()
    print(f"💾 Memory Usage:")
    print(f"  Process: {mem.get('process_rss_gb', 'N/A'):.2f} GB")
    print(f"  System: {mem.get('system_used_percent', 'N/A'):.1f}% used")
    if 'gpu_allocated_gb' in mem:
        print(f"  GPU: {mem['gpu_allocated_gb']:.2f} GB allocated")

def free_memory():
    """Free unused memory."""
    before = get_memory_usage().get('process_rss_gb', 0)
    clear_memory()
    after = get_memory_usage().get('process_rss_gb', 0)
    print(f"🧹 Freed {before - after:.2f} GB")

def download_artifacts():
    """Download all artifacts as a zip file (Colab only)."""
    if not IN_COLAB:
        print(f"📁 Artifacts at: {RUN_DIR}")
        return
    
    import shutil
    zip_path = f"/content/artifacts_{RUN_ID}.zip"
    shutil.make_archive(zip_path.replace('.zip', ''), 'zip', str(RUN_DIR))
    
    from google.colab import files
    files.download(zip_path)
    print(f"📦 Downloaded: artifacts_{RUN_ID}.zip")

def restart_runtime():
    """Restart Colab runtime to free memory."""
    if IN_COLAB:
        import os
        os.kill(os.getpid(), 9)

print("🛠️ Utility functions available:")
print("  - show_memory(): Display memory usage")
print("  - free_memory(): Free unused memory")
print("  - download_artifacts(): Download all run artifacts")
print("  - restart_runtime(): Restart Colab runtime")